In [1]:
import sys
import os
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
from statistics import mean
thisDir = os.path.expanduser('~/Desktop/Porteretal_taskprediction/')


outDir = thisDir + 'output/results/Ridge/'
taskList=['mem','motor','semantic','glass']

ALL=pd.read_csv(outDir+'ALL_Binary/acc.csv')
ALL.drop(columns=[ 'SS_tpv',
 'SS_rpv', 'OS_tpv','train_sub',
 'test_sub',
 'OS_rpv'],inplace=True)
ALL.rename(columns={'same_sub':'SS','diff_sub':'OS'},inplace=True)
ALL=pd.melt(ALL,value_vars=['SS','OS'],var_name='Analysis',value_name='acc')
Diff=ALL[ALL['Analysis']=='SS'].acc.values-ALL[ALL['Analysis']=='OS'].acc.values
TrueDiff=mean(Diff)


In [16]:
TrueDiff

0.2498288102837154

In [29]:

def shuffle(df, n=1, axis=0):     
    df['new']=np.random.permutation(df['acc'].values)
    null=df[df['Analysis']=='SS'].new.values-df[df['Analysis']=='OS'].new.values
    value=mean(null)
    return value


In [4]:
distribution=[]
for i in range(1000):
    null=shuffle(ALL)
    distribution.append(null)
null=pd.DataFrame(distribution,columns=['distribution'])

keep=null[null['distribution']>TrueDiff]
count=keep['distribution'].count()
p=(count+1)/1001
p

In [11]:
DS=pd.read_csv(outDir+'DS/acc.csv')
DS.drop(columns=['TPV','RPV','spec','sen'],inplace=True)
DS.rename(columns={'acc':'OS'},inplace=True)
#CV
CV=pd.read_csv(outDir+'CV/acc.csv')
CV.drop(columns=['TPV','RPV','spec','sen'],inplace=True)
CV.rename(columns={'acc':'SS'},inplace=True)
#permutation testing CV DS
DS.rename(columns={'train_sub':'sub'},inplace=True)
same_task=CV.merge(DS,how='left',on=['task','sub'])

Same=pd.melt(same_task,value_vars=['SS','OS'],var_name='Analysis',value_name='acc')
Diff=Same[Same['Analysis']=='SS'].acc.values-Same[Same['Analysis']=='OS'].acc.values
TrueDiff=mean(Diff)

In [12]:
distribution=[]
for i in range(1000):
    null=shuffle(Same)
    distribution.append(null)
null=pd.DataFrame(distribution,columns=['distribution'])

keep=null[null['distribution']>TrueDiff]
count=keep['distribution'].count()
p=(count+1)/1001
p

0.000999000999000999

In [16]:
MC=pd.read_csv(outDir+'ALL/multiclass_acc.csv',usecols=['train','same_sub','diff_sub'])
MC.rename(columns={'same_sub':'SS','diff_sub':'OS'},inplace=True)
Same=pd.melt(MC,value_vars=['SS','OS'],var_name='Analysis',value_name='acc')
Diff=Same[Same['Analysis']=='SS'].acc.values-Same[Same['Analysis']=='OS'].acc.values
TrueDiff=mean(Diff)
distribution=[]
for i in range(1000):
    null=shuffle(Same)
    distribution.append(null)
null=pd.DataFrame(distribution,columns=['distribution'])

keep=null[null['distribution']>TrueDiff]
count=keep['distribution'].count()
p=(count+1)/1001
p

0.000999000999000999

In [18]:
IndNet=pd.read_csv(outDir+'ALL/IndNet/acc.csv',usecols=['cv_acc','acc'])
IndNet.rename(columns={'cv_acc':'SS','acc':'OS'},inplace=True)
Same=pd.melt(IndNet,value_vars=['SS','OS'],var_name='Analysis',value_name='acc')
Diff=Same[Same['Analysis']=='SS'].acc.values-Same[Same['Analysis']=='OS'].acc.values
TrueDiff=mean(Diff)
distribution=[]
for i in range(1000):
    null=shuffle(Same)
    distribution.append(null)
null=pd.DataFrame(distribution,columns=['distribution'])

keep=null[null['distribution']>TrueDiff]
count=keep['distribution'].count()
p=(count+1)/1001
p

0.000999000999000999

In [19]:
TrueDiff

0.19392857142857142

In [3]:
#Difference score CVmDS permutation
pST=pd.read_csv(thisDir+'output/results/permutation/CVmDS_acc.csv',na_values=[9999])


DS=pd.read_csv(outDir+'DS/acc.csv')
#CV
CV=pd.read_csv(outDir+'CV/acc.csv')
#permutation testing CV DS
DS.rename(columns={'train_sub':'sub'},inplace=True)
same_task=CV.merge(DS,how='left',on=['task','sub'],suffixes=('','_DS'))
#SS-OS
same_task['diff'] =same_task.acc-same_task.acc_DS 
#subset so we only average the SS-OS per train/test tasks
ST=same_task[['task','diff']]
#take average
diffST=ST.groupby(['task']).mean()
diffST.reset_index(inplace=True)




#CVmDS
#dataframe of pvals for each task in same task 
#diffST.set_index('task',inplace=True)
tasks=[]
p_val=[]

for task in taskList:
    muST=diffST.loc[diffST['task']==task]
    muST=muST.iloc[0,1]
    ST_perm=pST.loc[pST['task']==task]
    ST_perm_keep=ST_perm[ST_perm['diff']>muST]
    count=ST_perm_keep['diff'].count()
    p=(count+1)/1001
    tasks.append(task)
    p_val.append(p)
ST_all=pd.DataFrame({'task':tasks, 'pval':p_val})
ST_all.set_index('task',inplace=True)
ST_all

,pval
task,
mem,0.000999
motor,0.000999
semantic,0.000999
glass,0.000999


In [8]:
#permutation testing SS BS
#permuted different task (SS-OS) SS-BS
pDT=pd.read_csv(thisDir+'output/results/permutation/SSmBS_acc.csv',na_values=[9999])
#prepping BS and SS
SS=pd.read_csv(outDir+'SS/acc.csv')
BS=pd.read_csv(outDir+'BS/acc.csv')
SS.drop(columns=['spec','sen'],inplace=True)
BS.drop(columns=['spec','sen'],inplace=True)
BS.rename(columns={'train_sub':'sub'},inplace=True)

diff_task=SS.merge(BS,how='left',on=['train_task','test_task','sub'],suffixes=('','_BS'))
#SS-OS
diff_task['diff'] =diff_task.acc-diff_task.acc_BS 
#subset so we only average the SS-OS per train/test tasks
DT=diff_task[['train_task','test_task','diff']]
#take average
diffDT=DT.groupby(['train_task','test_task']).mean()
diffDT.reset_index(inplace=True)
diffDT['TrTe'] = diffDT.train_task.map(str) + "_" + diffDT.test_task
DT=diffDT[['TrTe','diff']]
DT.set_index('TrTe',inplace=True)
"""
SS=pd.read_csv(outDir+'SS/acc.csv')
BS=pd.read_csv(outDir+'BS/acc.csv')
SS.drop(columns=['spec','sen'],inplace=True)
SS=SS.groupby(['train_task','test_task']).mean()
SS.reset_index(inplace=True)
#subset so we only average train/test tasks

#take average
BS.drop(columns=['spec','sen'],inplace=True)
BS=BS.groupby(['train_task','test_task']).mean()
BS.reset_index(inplace=True)

diffTask=SS.merge(BS,how='left',on=['train_task','test_task'],suffixes=('','_BS'))
diffTask['diff']=diffTask['acc']-diffTask['acc_BS']
diffTask['TrTe'] = diffTask.train_task.map(str) + "_" + diffTask.test_task
DT=diffDT[['TrTe','diff']]
DT.set_index('TrTe',inplace=True)
#DT=diffTask[['train_task','test_task','diff']]
#DT.set_index('train_task',inplace=True)
#dataframe to hold p values for each task by task
"""
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            DTmu=DT.loc[train_task+'_'+test_task].values
            DT_perm=pDT.loc[(pDT['train_task']==train_task) & (pDT['test_task']==test_task)]
            DT_perm_kept=DT_perm[DT_perm['diff']>DTmu[0]]
            count=DT_perm_kept['diff'].count()
            pval=(count+1)/(1001)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
DT_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
DT_all

,train_task,test_task,pval
0,mem,motor,0.043956
1,mem,semantic,0.038961
2,mem,glass,0.008991
3,motor,mem,0.014985
4,motor,semantic,0.034965
5,motor,glass,0.002997
6,semantic,mem,0.000999
7,semantic,motor,0.000999
8,semantic,glass,0.000999
9,glass,mem,0.000999


In [108]:
#permutation testing SS
###############################
###############################
#permuted labels SS includes task and rest
SS_permuted=pd.read_csv(thisDir+'output/results/permutation/SS/acc.csv',na_values=[9999])
#actual scores SS 
SS=pd.read_csv(outDir+'SS/acc.csv')
#subset so we only average train/test tasks
SS=SS[['train_task','test_task','acc']]
#take average
avg=SS.groupby(['train_task', 'test_task']).mean()
avg.reset_index(inplace=True)
avg['TrTe'] = avg.train_task.map(str) + "_" + avg.test_task
avg=avg[['TrTe','acc']]
avg.set_index('TrTe',inplace=True)

#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            Scmu=avg.loc[train_task+'_'+test_task].values
            SS_p=SS_permuted.loc[(SS_permuted['train_task']==train_task) & (SS_permuted['test_task']==test_task)]
            SS_p_kept=SS_p[SS_p['acc']>Scmu[0]]
            count=SS_p_kept['acc'].count()
            pval=(count+1)/(1001)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
SS_p_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
SS_p_all

,train_task,test_task,pval
0,mem,motor,0.002997
1,mem,semantic,0.008991
2,mem,glass,0.010989
3,motor,mem,0.092907
4,motor,semantic,0.095904
5,motor,glass,0.059940
6,semantic,mem,0.023976
7,semantic,motor,0.061938
8,semantic,glass,0.007992
9,glass,mem,0.011988


In [80]:
#permutation testing SS TASK ONLY
###############################
###############################
#permuted labels SS
SS_permuted=pd.read_csv(thisDir+'output/results/permutation/SS/sepTask_perm.csv',na_values=[9999])
#actual scores SS 
SS=pd.read_csv(thisDir+'output/results/permutation/SS/sepTask_Trueacc.csv') #difference scores 
#subset so we only average train/test tasks
SS=SS[['train_task','test_task','diff']]
#take average
avg=SS.groupby(['train_task', 'test_task']).mean()
avg.reset_index(inplace=True)
avg['TrTe'] = avg.train_task.map(str) + "_" + avg.test_task
avg=avg[['TrTe','diff']]
avg.set_index('TrTe',inplace=True)

#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            Scmu=avg.loc[train_task+'_'+test_task].values
            SS_p=SS_permuted.loc[(SS_permuted['train_task']==train_task) & (SS_permuted['test_task']==test_task)]
            SS_p_kept=SS_p[SS_p['diff']>Scmu[0]]
            count=SS_p_kept['diff'].count()
            pval=(count+1)/(8001)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
SS_p_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
SS_p_all

,train_task,test_task,pval
0,mem,motor,0.321210
1,mem,semantic,0.407074
2,mem,glass,0.409199
3,motor,mem,0.060242
4,motor,semantic,0.051619
5,motor,glass,0.061492
6,semantic,mem,0.104612
7,semantic,motor,0.075116
8,semantic,glass,0.088489
9,glass,mem,0.417198


In [60]:
#permutation testing SS TASK ONLY
###############################
###############################
#permuted labels SS

SS_permuted=pd.read_csv(thisDir+'output/results/permutation/SS/acc.csv',na_values=[9999])
#actual scores SS 
SS=pd.read_csv(outDir+'SS/separate_broken_acc.csv')
#subset so we only average train/test tasks
SS=SS[['train_task','test_task','Different Task']]
#take average
avg=SS.groupby(['train_task', 'test_task']).mean()
avg.reset_index(inplace=True)
avg['TrTe'] = avg.train_task.map(str) + "_" + avg.test_task
avg=avg[['TrTe','Different Task']]
avg.set_index('TrTe',inplace=True)

#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            Scmu=avg.loc[train_task+'_'+test_task].values
            SS_p=SS_permuted.loc[(SS_permuted['train_task']==train_task) & (SS_permuted['test_task']==test_task)]
            SS_p_kept=SS_p[SS_p['diff']>Scmu[0]]
            count=SS_p_kept['diff'].count()
            pval=(count+1)/(8001)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
SS_p_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
SS_p_all

,train_task,test_task,pval
0,mem,motor,0.062242
1,mem,semantic,0.024622
2,mem,glass,0.018623
3,motor,mem,0.420697
4,motor,semantic,0.582177
5,motor,glass,0.421697
6,semantic,mem,0.308211
7,semantic,motor,0.556055
8,semantic,glass,0.276215
9,glass,mem,0.090489


In [109]:
BS_permuted=pd.read_csv(thisDir+'output/results/permutation/BS/acc.csv',na_values=[9999])
#actual scores
BS=pd.read_csv(outDir+'BS/acc.csv')
#subset so we only average train/test tasks
BS=BS[['train_task','test_task','acc']]
#take average
avg=BS.groupby(['train_task', 'test_task']).mean()
avg.reset_index(inplace=True)
avg['TrTe'] = avg.train_task.map(str) + "_" + avg.test_task
avg=avg[['TrTe','acc']]
avg.set_index('TrTe',inplace=True)

#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            Scmu=avg.loc[train_task+'_'+test_task].values
            BS_p=BS_permuted.loc[(BS_permuted['train_task']==train_task) & (BS_permuted['test_task']==test_task)]
            BS_p_kept=BS_p[BS_p['acc']>Scmu[0]]
            count=BS_p_kept['acc'].count()
            pval=(count+1)/(1009)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
BS_p_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
BS_p_all

,train_task,test_task,pval
0,mem,motor,0.010902
1,mem,semantic,0.003964
2,mem,glass,0.014866
3,motor,mem,0.111001
4,motor,semantic,0.114965
5,motor,glass,0.138751
6,semantic,mem,0.198216
7,semantic,motor,0.248761
8,semantic,glass,0.210109
9,glass,mem,0.155600


In [115]:
CV_permuted=pd.read_csv(thisDir+'output/results/permutation/CV/acc.csv',na_values=[9999])
#actual scores
CV=pd.read_csv(outDir+'CV/acc.csv')
#subset so we only average train/test tasks
CV=CV[['task','acc']]
#take average
avg=CV.groupby(['task']).mean()
#dataframe to hold p values for each task by task
training=[]
p_values=[]
for train_task in taskList:
    Scmu=avg.loc[train_task].values
    CV_p=CV_permuted.loc[(CV_permuted['task']==train_task)]
    CV_p_kept=CV_p[CV_p['acc']>Scmu[0]]
    count=CV_p_kept['acc'].count()
    pval=(count+1)/(1001)
    training.append(train_task)
    testing.append(test_task)
    p_values.append(pval)
CV_p_all=pd.DataFrame({'train_task':training,'pval':p_values})
CV_p_all

,train_task,pval
0,mem,0.000999
1,motor,0.000999
2,semantic,0.000999
3,glass,0.000999


In [5]:
DS_permuted=pd.read_csv(thisDir+'output/results/permutation/DS/acc.csv',na_values=[9999])
#actual scores
DS=pd.read_csv(outDir+'DS/acc.csv')
#subset so we only average train/test tasks
DS=DS[['task','acc']]
#take average
avg=DS.groupby(['task']).mean()
#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    Scmu=avg.loc[train_task].values
    DS_p=DS_permuted.loc[(DS_permuted['task']==train_task)]
    DS_p_kept=DS_p[DS_p['acc']>Scmu[0]]
    count=DS_p_kept['acc'].count()
    pval=(count+1)/(1009)
    training.append(train_task)
    #testing.append(test_task)
    p_values.append(pval)
DS_p_all=pd.DataFrame({'train_task':training,'pval':p_values})
DS_p_all

,train_task,pval
0,mem,0.003964
1,motor,0.005946
2,semantic,0.072349
3,glass,0.151635


In [6]:
DS_p

,train_sub,test_sub,task,acc
0,MSC01,MSC02,glass,0.500000
4,MSC01,MSC03,glass,0.535000
8,MSC01,MSC04,glass,0.670000
12,MSC01,MSC05,glass,0.500000
16,MSC01,MSC06,glass,0.542105
20,MSC01,MSC07,glass,0.455000
24,MSC01,MSC10,glass,0.415000
28,MSC02,MSC01,glass,0.670000
32,MSC02,MSC03,glass,0.665000
36,MSC02,MSC04,glass,0.390000


In [12]:
#testing sensitivity CVmSS 
#this procedure combines performance of the testing task so the combined plot I can test for significance
#permutation 
CVmSS_permuted=pd.read_csv(thisDir+'output/results/permutation/STmDT_CVmSS_acc.csv',na_values=[9999])

#actual scores SS 
CV=pd.read_csv(outDir+'CV/acc.csv')
SS=pd.read_csv(outDir+'SS/acc.csv')
SS.drop(columns=['spec','sen'])
SS=SS.groupby(['train_task']).mean()
SS.reset_index(inplace=True)
#subset so we only average train/test tasks
CV=CV[['task','acc']]
#take average
avgCV=CV.groupby(['task']).mean()
avgCV.reset_index(inplace=True)
avgCV.rename(columns={'task':'train_task'},inplace=True)

diffTask=avg.merge(avgCV,how='left',on=['train_task'],suffixes=('','_CV'))
diffTask['diff']=diffTask['acc_CV']-diffTask['acc']
avg=diffTask[['train_task','diff']]
avg.set_index('train_task',inplace=True)


In [15]:
taskList=['mem','motor','semantic','glass']
#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    Scmu=avg.loc[train_task].values
    btw=CVmSS_permuted.loc[(CVmSS_permuted['train_task']==train_task)]
    btw_kept=btw[btw['diff']>Scmu[0]]
    count=btw_kept['diff'].count()
    pval=(count+1)/(3001) #since keeping all testing tasks included 
    training.append(train_task)
    p_values.append(pval)
btw_all=pd.DataFrame({'train_task':training,'pval':p_values})
btw_all

,train_task,pval
0,mem,0.034322
1,motor,0.000333
2,semantic,0.005332
3,glass,0.625125


In [33]:
#testing sensitivity CVmSS
#permutation 
CVmSS_permuted=pd.read_csv(thisDir+'output/results/permutation/STmDT_CVmSS_acc.csv',na_values=[9999])

#actual scores SS 
CV=pd.read_csv(outDir+'CV/acc.csv')
SS=pd.read_csv(outDir+'SS/acc.csv')
#subset so we only average train/test tasks
SS=SS[['train_task','test_task','acc']]
#take average
avg=SS.groupby(['train_task', 'test_task']).mean()
avg.reset_index(inplace=True)

#subset so we only average train/test tasks
CV=CV[['task','acc']]
#take average
avgCV=CV.groupby(['task']).mean()
avgCV.reset_index(inplace=True)
avgCV.rename(columns={'task':'train_task'},inplace=True)

diffTask=avg.merge(avgCV,how='left',on=['train_task'],suffixes=('','_CV'))
diffTask['diff']=diffTask['acc_CV']-diffTask['acc']
diffTask['TrTe'] = diffTask.train_task.map(str) + "_" + diffTask.test_task
avg=diffTask[['TrTe','diff']]
avg.set_index('TrTe',inplace=True)

taskList=['mem','motor','semantic','glass']
#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            Scmu=avg.loc[train_task+'_'+test_task].values
            btw=CVmSS_permuted.loc[(CVmSS_permuted['train_task']==train_task) & (CVmSS_permuted['test_task']==test_task)]
            btw_kept=btw[btw['diff']>Scmu[0]]
            count=btw_kept['diff'].count()
            pval=(count+1)/(1001)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
btw_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
btw_all
######################################################
#permutation test for CVmSS testing task specificity##
######################################################

,train_task,test_task,pval
0,mem,motor,0.034965
1,mem,semantic,0.066933
2,mem,glass,0.012987
3,motor,mem,0.000999
4,motor,semantic,0.000999
5,motor,glass,0.000999
6,semantic,mem,0.000999
7,semantic,motor,0.000999
8,semantic,glass,0.024975
9,glass,mem,0.181818


In [3]:
#testing sensitivity CVmSS
#permutation 
CVmSS_permuted=pd.read_csv(thisDir+'output/results/permutation/TaskOnly_STmDT_CVmSS_acc.csv',na_values=[9999])

CVmSS_True=pd.read_csv(thisDir+'output/results/permutation/TrueDiff_TaskOnly_STmDT_CVmSS_acc.csv',na_values=[9999])

In [33]:
taskList=['mem','motor','semantic','glass']
#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            Scmu=CVmSS_True[(CVmSS_True['train_task']==train_task) & (CVmSS_True['test_task']==test_task)]
            btw=CVmSS_permuted.loc[(CVmSS_permuted['train_task']==train_task) & (CVmSS_permuted['test_task']==test_task)]
            btw_kept=btw[btw['diff']>Scmu.iloc[0,2]]
            count=btw_kept['diff'].count()
            pval=(count+1)/(1001)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
btw_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
btw_all
############################################################################
#permutation test for CVmSS testing task specificity TASK ONLY Permutation##
############################################################################

,train_task,test_task,pval
0,mem,motor,0.079920
1,mem,semantic,0.013986
2,mem,glass,0.017982
3,motor,mem,0.000999
4,motor,semantic,0.000999
5,motor,glass,0.000999
6,semantic,mem,0.000999
7,semantic,motor,0.000999
8,semantic,glass,0.000999
9,glass,mem,0.011988


In [34]:
btw

,train_task,test_task,diff
2,glass,semantic,0.017778
14,glass,semantic,0.203750
26,glass,semantic,-0.091806
38,glass,semantic,-0.100833
50,glass,semantic,-0.106111
...,...,...,...
11942,glass,semantic,0.018194
11954,glass,semantic,-0.020000
11966,glass,semantic,0.051528
11978,glass,semantic,-0.079444


In [72]:
#Calculating SD and Mean for all regular analysis
def statsACC(df,analysis):
    if analysis=='CV':
        print('cross validation stats')
        stats=df.groupby('task').mean()
        stats.rename(columns={'acc':'Mean'},inplace=True)
        sd=df.groupby('task').std()
        stats['Std']=sd['acc']
        stats.to_csv(outDir+analysis+'/stats.csv', index=True)
    elif analysis=='SS':
        print('same sub stats')
        df.drop(['sub'], axis=1, inplace=True)
        stats=df.groupby(['test_task', 'train_task']).mean()
        stats.rename(columns={'acc':'Mean'}, inplace=True)
        sd=df.groupby(['test_task', 'train_task']).std()
        stats['Std']=sd['acc']
        stats.reset_index(inplace=True)
        stats.to_csv(outDir+analysis+'/stats.csv', index=True)
    elif analysis=='DS':
        print('diff sub stats')
        df.drop(['train_sub', 'test_sub'], axis=1, inplace=True)
        stats=df.groupby('task').mean()
        stats.rename(columns={'acc':'Mean'}, inplace=True)
        sd=df.groupby('task').std()
        stats['Std']=sd['acc']
        stats.to_csv(outDir+analysis+'/stats.csv', index=True)
    elif analysis=='BS':
        print('diff sub diff task stats')
        df.drop(['train_sub', 'test_sub'], axis=1, inplace=True)
        stats=df.groupby(['test_task', 'train_task']).mean()
        stats.rename(columns={'acc':'Mean'}, inplace=True)
        sd=df.groupby(['test_task', 'train_task']).std()
        stats['Std']=sd['acc']
        stats.reset_index(inplace=True)
        stats.to_csv(outDir+analysis+'/stats.csv', index=True)
    return stats

In [79]:
#statsACC(CV,'CV')
statsACC(SS,'SS')
#statsACC(DS,'DS')
#statsACC(BS,'BS')


same sub stats


,test_task,train_task,Mean,Std
0,glass,mem,0.841806,0.141014
1,glass,motor,0.658750,0.148246
2,glass,semantic,0.807083,0.101249
3,mem,glass,0.760694,0.139531
4,mem,motor,0.632500,0.103957
5,mem,semantic,0.694375,0.123446
6,motor,glass,0.792569,0.142429
7,motor,mem,0.876250,0.120971
8,motor,semantic,0.628750,0.093417
9,semantic,glass,0.900069,0.073824


In [82]:
ALL['cv_acc'].mean()
ALL['cv_acc'].std()

0.010808903106404302

In [84]:
#ALL['acc'].mean()
ALL['acc'].std()

0.11435408419187446

In [26]:
SS=pd.read_csv(outDir+'SS/acc.csv')

In [30]:
SS.groupby(['train_task']).mean()

,acc,spec,sen
train_task,,,
glass,0.858125,0.986250,0.730000
mem,0.894560,0.987731,0.801389
motor,0.698426,0.999583,0.397269
semantic,0.773403,0.999583,0.547222


In [22]:
SS.groupby(['train_task','test_task']).mean()

acc      spec       sen
train_task test_task                              
glass      mem        0.830139  0.986250  0.674028
           motor      0.808611  0.973750  0.643472
           semantic   0.935625  0.998750  0.872500
mem        glass      0.878681  0.984444  0.772917
           motor      0.888125  0.983750  0.792500
           semantic   0.916875  0.995000  0.838750
motor      glass      0.738403  1.000000  0.476806
           mem        0.693750  0.998750  0.388750
           semantic   0.663125  1.000000  0.326250
semantic   glass      0.844583  1.000000  0.689167
           mem        0.772500  0.998750  0.546250
           motor      0.703125  1.000000  0.406250

In [23]:
SS.groupby(['train_task','test_task']).std()

acc      spec       sen
train_task test_task                              
glass      mem        0.114660  0.038891  0.206827
           motor      0.153236  0.066534  0.262088
           semantic   0.051508  0.003536  0.102365
mem        glass      0.108183  0.016211  0.210060
           motor      0.086104  0.034200  0.176857
           semantic   0.080664  0.007559  0.164007
motor      glass      0.164101  0.000000  0.328202
           mem        0.156610  0.003536  0.312796
           semantic   0.142326  0.000000  0.284652
semantic   glass      0.097304  0.000000  0.194608
           mem        0.143203  0.003536  0.286752
           motor      0.120947  0.000000  0.241894

In [8]:
CV.groupby(['task']).mean()

,acc,spec,sen
task,,,
glass,0.878472,1.0,0.756944
mem,0.993750,1.0,0.987500
motor,0.981250,1.0,0.962500
semantic,0.943750,1.0,0.887500


In [9]:
CV.groupby(['task']).std()

,acc,spec,sen
task,,,
glass,0.064065,0.0,0.128131
mem,0.017678,0.0,0.035355
motor,0.025877,0.0,0.051755
semantic,0.032043,0.0,0.064087


In [11]:
DS.groupby(['task']).mean()

,acc,spec,sen
task,,,
glass,0.577480,0.996071,0.158889
mem,0.894375,0.821964,0.966786
motor,0.788393,0.843750,0.733036
semantic,0.669911,0.998214,0.341607


In [12]:
DS.groupby(['task']).std()

,acc,spec,sen
task,,,
glass,0.101508,0.014607,0.208554
mem,0.100476,0.210743,0.067017
motor,0.154729,0.282261,0.311407
semantic,0.144881,0.005430,0.291627


In [14]:
BS.groupby(['train_task','test_task']).mean()

acc      spec       sen
train_task test_task                              
glass      mem        0.571042  0.996071  0.146012
           motor      0.584782  0.996071  0.173492
           semantic   0.577837  0.996071  0.159603
mem        glass      0.764921  0.821726  0.708115
           motor      0.821071  0.821964  0.820179
           semantic   0.790804  0.821964  0.759643
motor      glass      0.605476  0.843770  0.367183
           mem        0.620179  0.843750  0.396607
           semantic   0.616518  0.843750  0.389286
semantic   glass      0.555040  0.998214  0.111865
           mem        0.562411  0.998214  0.126607
           motor      0.553036  0.998214  0.107857

In [16]:
BS.groupby(['train_task','test_task']).std()

acc      spec       sen
train_task test_task                              
glass      mem        0.114925  0.014607  0.235733
           motor      0.102336  0.014607  0.212573
           semantic   0.094114  0.014607  0.191819
mem        glass      0.131100  0.210683  0.243705
           motor      0.121392  0.210743  0.197581
           semantic   0.118036  0.210743  0.192986
motor      glass      0.098486  0.282237  0.338109
           mem        0.114638  0.282261  0.376767
           semantic   0.116396  0.282261  0.356059
semantic   glass      0.087032  0.005430  0.175022
           mem        0.095594  0.005430  0.192347
           motor      0.085810  0.005430  0.171620

In [32]:
import pandas as pd

rest=pd.read_csv('~/Desktop/rest.csv')

rest['min']=(rest['rest']*2.2)/60

#rest=rest[rest['min']>30]

In [33]:
rest['half']=rest['min']/2
rest['fourth']=rest['min']/4

In [34]:
rest

,sub,rest,min,half,fourth
0,MSC01,523,19.176667,9.588333,4.794167
1,MSC01,717,26.290000,13.145000,6.572500
2,MSC01,732,26.840000,13.420000,6.710000
3,MSC01,778,28.526667,14.263333,7.131667
4,MSC01,710,26.033333,13.016667,6.508333
...,...,...,...,...,...
75,MSC10,336,12.320000,6.160000,3.080000
76,MSC10,545,19.983333,9.991667,4.995833
77,MSC10,526,19.286667,9.643333,4.821667
78,MSC10,766,28.086667,14.043333,7.021667


In [21]:
#permutation comparing ALL performance with permuted ALL

ALL=pd.read_csv(outDir+'ALL/LOO_acc.csv')
ALL_perm=pd.read_csv(thisDir+'output/results/permutation/ALL_acc.csv',na_values=[9999])
ALL_perm.head()
ALL['diff']=ALL['cv_acc']-ALL['acc']
ALL_avg=ALL['diff'].mean()

In [22]:
ALL.mean()

cv_acc     0.981510
cv_sen     0.996875
cv_spec    0.966146
cv_tpv     0.974884
cv_rpv     0.997500
acc        0.731682
ds_sen     0.949001
ds_spec    0.518114
ds_tpv     0.698783
ds_rpv     0.936035
diff       0.249829
dtype: float64

In [23]:
ALL.std()

cv_acc     0.011730
cv_sen     0.008343
cv_spec    0.019444
cv_tpv     0.013134
cv_rpv     0.006674
acc        0.135279
ds_sen     0.074320
ds_spec    0.307863
ds_tpv     0.148045
ds_rpv     0.076056
diff       0.137381
dtype: float64

In [3]:
ALL_avg=ALL['diff'].mean()
perm_kept=ALL_perm[ALL_perm['diff_acc']>ALL_avg]
count=perm_kept['diff_acc'].count()
pval=(count+1)/(1001)

In [4]:
pval

0.000999000999000999

In [5]:
ALL_cv_avg=ALL['cv_acc'].mean()
ALL_ds_avg=ALL['acc'].mean()

In [6]:
perm_kept=ALL_perm[ALL_perm['SS']>ALL_cv_avg]
count=perm_kept['SS'].count()
pval=(count+1)/(1001)

In [7]:
pval

0.000999000999000999

In [8]:
perm_kept=ALL_perm[ALL_perm['OS']>ALL_ds_avg]
count=perm_kept['OS'].count()
pval=(count+1)/(1001)

In [9]:
pval

0.000999000999000999

In [73]:
sixty_res.head()

,Days,Analysis,acc
0,30,Same Subject,0.966667
1,30,Same Subject,0.966667
2,30,Same Subject,0.966667
3,30,Same Subject,0.966667
4,30,Same Subject,0.966667


In [64]:
sixty_res=pd.read_csv(thisDir+'output/results/permutation/ALL/manDays.csv')
sixty_res=sixty_res[['diff_sub','Days']]

In [65]:
five=sixty_res[sixty_res['Days']==8]

ten=sixty_res[sixty_res['Days']==20]

thirty=sixty_res[sixty_res['Days']==36]


In [66]:
five['diff_sub'].mean()

0.5994848384701407

In [67]:
ten['diff_sub'].mean()

0.676516588165832

In [68]:
thirty['diff_sub'].mean()

0.7093930144052575

In [94]:
#BS['acc'].mean()
BS['acc'].std()

0.1420709585872698

In [95]:
SS['acc'].mean()
SS['acc'].std()

0.14540192001446242

In [23]:
#permutation testing SS BS for condensed plot
#permuted different task (SS-OS) SS-BS
pDT=pd.read_csv(thisDir+'output/results/permutation/SSmBS_acc.csv',na_values=[9999])
#prepping BS and SS


#actual scores SS 
SS=pd.read_csv(outDir+'SS/acc.csv')
BS=pd.read_csv(outDir+'BS/acc.csv')
SS.drop(columns=['spec','sen'],inplace=True)
SS=SS.groupby(['train_task']).mean()
SS.reset_index(inplace=True)
#subset so we only average train/test tasks

#take average
BS.drop(columns=['spec','sen'],inplace=True)
BS=BS.groupby(['train_task']).mean()
BS.reset_index(inplace=True)

diffTask=SS.merge(BS,how='left',on=['train_task'],suffixes=('','_BS'))


diffTask['diff']=diffTask['acc']-diffTask['acc_BS']
avg=diffTask[['train_task','diff']]
avg.set_index('train_task',inplace=True)

#dataframe to hold p values for each task by task
training=[]
p_values=[]
for train_task in taskList:
    DTmu=avg.loc[train_task].values
    DT_perm=pDT.loc[(pDT['train_task']==train_task)]
    DT_perm_kept=DT_perm[DT_perm['diff']>DTmu[0]]
    count=DT_perm_kept['diff'].count()
    pval=(count+1)/(3001)#since keeping all 3 testing tasks
    training.append(train_task)
    testing.append(test_task)
    p_values.append(pval)
DT_all=pd.DataFrame({'train_task':training,'pval':p_values})
DT_all

,train_task,pval
0,mem,0.026658
1,motor,0.010663
2,semantic,0.000333
3,glass,0.000333


In [19]:
DT_perm=pDT.loc[(pDT['train_task']=='mem')]

In [55]:
ALL.cv_acc.mean()-ALL.acc.mean()

0.24929209183673473

In [62]:
CV[CV['task']=='glass'].acc.mean()-DS[DS['task']=='glass'].acc.mean()

0.30099206349206364

In [63]:
CV[CV['task']=='semantic'].acc.mean()-DS[DS['task']=='semantic'].acc.mean()

0.27383928571428584

In [64]:
CV[CV['task']=='motor'].acc.mean()-DS[DS['task']=='motor'].acc.mean()

0.1562819664902999

In [65]:
CV[CV['task']=='mem'].acc.mean()-DS[DS['task']=='mem'].acc.mean()

0.09727678571428611

In [67]:
DS.groupby('task').mean()

,acc,spec,sen
task,,,
glass,0.577480,0.996071,0.158889
mem,0.888661,0.868929,0.908393
motor,0.824968,0.991270,0.658666
semantic,0.669911,0.998214,0.341607


In [24]:
MC_permuted=pd.read_csv(thisDir+'output/results/permutation/ALL/multiclass_perm.csv',na_values=[9999])
MC_True=pd.read_csv(thisDir+'output/results/permutation/ALL/multiclass_acc.csv',na_values=[9999])
#just regular permutation test
MC_CV_avg=MC_True['same'].mean()
perm_kept=MC_permuted[MC_permuted['same']>MC_CV_avg]
count=perm_kept['same'].count()
pval=(count+1)/(8001)
pval



0.00012498437695288088

In [12]:
MC_CV_avg=MC_True['diff'].mean()
perm_kept=MC_permuted[MC_permuted['diff']>MC_CV_avg]
count=perm_kept['diff'].count()
pval=(count+1)/(8001)
pval

0.00012498437695288088

In [3]:
MC_permuted=pd.read_csv(thisDir+'output/results/permutation/ALL/multiclass_Diffperm.csv',na_values=[9999])
#difference permutation test
MC_True=pd.read_csv(thisDir+'output/results/permutation/ALL/TrueDiff_multiclass_acc.csv',na_values=[9999])

In [4]:
True_avg=MC_True['diff'].mean()
perm_kept=MC_permuted[MC_permuted['diff']>True_avg]
count=perm_kept['diff'].count()
pval=(count+1)/(8001)
pval
############################################################################
#Multiclass Permutation##
############################################################################

0.00012498437695288088

In [97]:
DS=pd.read_csv(thisDir+'output/results/acc/DS/acc.csv')
MC=pd.read_csv(thisDir+'output/results/acc/ALL/multiclass_acc.csv')
CV=pd.read_csv(thisDir+'output/results/acc/CV/acc.csv')
ALL=pd.read_csv(thisDir+'output/results/acc/ALL/LOO_acc.csv')

In [98]:
MC.mean()

same_sub    0.834583
diff_sub    0.536237
MCC_CV      0.823671
MCC_DS      0.471158
rest_CV     0.910417
mem_CV      0.875347
sem_CV      0.729977
mot_CV      0.475926
glass_CV    0.937558
rest_DS     0.644351
mem_DS      0.626727
sem_DS      0.306431
mot_DS      0.043998
glass_DS    0.645258
dtype: float64

In [102]:
DS.groupby(['task']).mean()

,acc,spec,sen,TPV,RPV
task,,,,,
glass,0.580237,0.996071,0.158889,0.986706,0.556626
mem,0.888078,0.871964,0.908393,0.897792,0.935767
motor,0.827648,0.991448,0.658666,0.990022,0.786110
semantic,0.669911,0.998214,0.341607,0.997103,0.627119


In [101]:
DS.groupby(['task', 'train_sub']).mean()

acc      spec       sen       TPV       RPV
task     train_sub                                                  
glass    MSC01      0.618045  1.000000  0.228571  1.000000  0.592212
         MSC02      0.520902  1.000000  0.034286       NaN  0.515035
         MSC03      0.518759  1.000000  0.030000  1.000000  0.511696
         MSC04      0.700489  0.987143  0.411746  0.971905  0.641456
         MSC05      0.574398  1.000000  0.142698  1.000000  0.545707
         MSC06      0.553175  1.000000  0.106349       NaN  0.536598
         MSC07      0.513759  1.000000  0.020000  1.000000  0.509190
         MSC10      0.642368  0.981429  0.297460  0.975926  0.601115
mem      MSC01      0.877063  0.780000  0.981429  0.834971  0.982987
         MSC02      0.931587  0.885714  0.984286  0.909325  0.985714
         MSC03      0.758036  1.000000  0.516071  1.000000  0.706319
         MSC04      0.921429  0.872857  0.977143  0.900174  0.979394
         MSC05      0.898492  0.887143  0.908571  0.914578  0.923395
         MSC06      0.945952  0.914286  0.979643  0.936530  0.979293
         MSC07      0.900238  0.828571  0.978571  0.872025  0.981602
         MSC10      0.871825  0.807143  0.941429  0.843932  0.953650
motor    MSC01      0.919436  0.977143  0.858254  0.973831  0.900289
         MSC02      0.573609  1.000000  0.133016  1.000000  0.547453
         MSC03      0.846165  1.000000  0.686984  1.000000  0.787563
         MSC04      0.969398  0.984286  0.953333  0.985584  0.962021
         MSC05      0.762256  0.997143  0.520000  0.995578  0.694572
         MSC06      0.797243  1.000000  0.593651  1.000000  0.748003
         MSC07      0.877360  0.987302  0.764727  0.987029  0.819763
         MSC10      0.875714  0.985714  0.759365  0.985815  0.829218
semantic MSC01      0.739286  0.997143  0.481429  0.995556  0.692437
         MSC02      0.537857  0.998571  0.077143  0.994444  0.523283
         MSC03      0.505714  1.000000  0.011429       NaN  0.503182
         MSC04      0.725000  1.000000  0.450000  1.000000  0.664999
         MSC05      0.748571  1.000000  0.497143  1.000000  0.684295
         MSC06      0.781429  0.991429  0.571429  0.992060  0.716831
         MSC07      0.552857  1.000000  0.105714  1.000000  0.530877
         MSC10      0.768571  0.998571  0.538571  0.998214  0.701044

In [87]:
DS.groupby('task').std()

,acc,spec,sen,TPV,RPV
task,,,,,
glass,0.100898,0.014607,0.208554,0.036296,0.082403
mem,0.101686,0.159253,0.186915,0.114905,0.111323
motor,0.140331,0.023899,0.292299,0.024979,0.150018
semantic,0.144881,0.005430,0.291627,0.006344,0.124482


In [100]:
DS

,train_sub,test_sub,task,acc,spec,sen,TPV,RPV
0,MSC01,MSC02,glass,0.565000,1.00,0.130000,NaN,0.535724
1,MSC01,MSC02,semantic,0.870000,0.98,0.760000,0.977778,0.809470
2,MSC01,MSC02,motor,0.925000,0.90,0.950000,0.902987,0.959231
3,MSC01,MSC02,mem,0.895000,0.79,1.000000,0.828089,1.000000
4,MSC01,MSC03,glass,0.515000,1.00,0.030000,NaN,0.508187
...,...,...,...,...,...,...,...,...
219,MSC10,MSC06,mem,0.975000,1.00,0.950000,1.000000,0.958741
220,MSC10,MSC07,glass,0.540000,0.98,0.100000,NaN,0.521655
221,MSC10,MSC07,semantic,0.690000,1.00,0.380000,1.000000,0.617647
222,MSC10,MSC07,motor,0.921053,1.00,0.833333,1.000000,0.871212


In [27]:
MC=pd.read_csv(thisDir+'output/results/acc/ALL/multiclass_acc.csv')

In [29]:
MC.mean()

same_sub    0.834583
diff_sub    0.536237
MCC_CV      0.823671
MCC_DS      0.471158
rest_CV     0.910417
mem_CV      0.875347
sem_CV      0.729977
mot_CV      0.475926
glass_CV    0.937558
rest_DS     0.644351
mem_DS      0.626727
sem_DS      0.306431
mot_DS      0.043998
glass_DS    0.645258
dtype: float64

In [30]:
MC.std()

same_sub    0.056285
diff_sub    0.074004
MCC_CV      0.060883
MCC_DS      0.075060
rest_CV     0.059052
mem_CV      0.086491
sem_CV      0.199328
mot_CV      0.114464
glass_CV    0.044858
rest_DS     0.095579
mem_DS      0.090989
sem_DS      0.187636
mot_DS      0.063408
glass_DS    0.176239
dtype: float64

In [37]:
DS=pd.read_csv(thisDir+'output/results/acc/DS/acc.csv')
CV=pd.read_csv(thisDir+'output/results/acc/CV/acc.csv')

In [35]:
DS.groupby('task').mean()

,acc,spec,sen,TPV,RPV
task,,,,,
glass,0.580237,0.996071,0.158889,0.986706,0.556626
mem,0.888078,0.871964,0.908393,0.897792,0.935767
motor,0.827648,0.991448,0.658666,0.990022,0.786110
semantic,0.669911,0.998214,0.341607,0.997103,0.627119


In [36]:
DS.groupby('task').std()

,acc,spec,sen,TPV,RPV
task,,,,,
glass,0.100898,0.014607,0.208554,0.036296,0.082403
mem,0.101686,0.159253,0.186915,0.114905,0.111323
motor,0.140331,0.023899,0.292299,0.024979,0.150018
semantic,0.144881,0.005430,0.291627,0.006344,0.124482


In [38]:
CV.groupby('task').mean()

,acc,spec,sen,TPV,RPV
task,,,,,
glass,0.878472,1.0,0.756944,1.0,0.814498
mem,0.985938,1.0,0.971875,1.0,0.977273
motor,0.981250,1.0,0.962500,1.0,0.965909
semantic,0.943750,1.0,0.887500,1.0,0.901515


In [39]:
CV.groupby('task').std()

,acc,spec,sen,TPV,RPV
task,,,,,
glass,0.064065,0.0,0.128131,0.0,0.076620
mem,0.026252,0.0,0.052504,0.0,0.042083
motor,0.025877,0.0,0.051755,0.0,0.047050
semantic,0.032043,0.0,0.064087,0.0,0.052486


In [40]:
SS=pd.read_csv(thisDir+'output/results/acc/SS/separate_broken_acc.csv')

In [85]:
SS.groupby(['train_task','test_task']).std()

diff
train_task test_task          
glass      mem        0.127827
           motor      0.202786
           semantic   0.096133
mem        glass      0.107539
           motor      0.207057
           semantic   0.078419
motor      glass      0.348991
           mem        0.343623
           semantic   0.241963
semantic   glass      0.202187
           mem        0.291307
           motor      0.289953

In [19]:
IndNet=pd.read_csv(outDir+'ALL/IndNet/acc.csv')
#permInd=pd.read_csv('/Users/Alexis/Desktop/MSC_Alexis/analysis/output/results/permutation/ALL/IndNet/permutation.csv')

In [21]:
IndNet['cv'].mean()

0.9166582321187585

In [22]:
CV=IndNet['cv'].mean()
DS=IndNet['ds'].mean()
perm_kept=permInd[permInd['SS']>CV]
count=perm_kept['SS'].count()
pval=(count+1)/(1001)

In [23]:
pval

0.000999000999000999

In [24]:
perm_kept=permInd[permInd['OS']>DS]
count=perm_kept['OS'].count()
pval=(count+1)/(1001)


In [25]:
pval

0.000999000999000999

In [2]:
perm_memPres=pd.read_csv('/Users/aporter1350/Desktop/Porteretal_taskprediction/output/results/Ridge/single_task/permutation_memPres.csv')
memPres=pd.read_csv('/Users/aporter1350/Desktop/Porteretal_taskprediction/output/results/Ridge/single_task/sep_mem_pres_accCG.csv',usecols=['diff_sub','same_sub'])

In [3]:
ds_mu=memPres['diff_sub'].mean()
ss_mu=memPres['same_sub'].mean()

In [16]:
perm_kept=perm_memPres[perm_memPres['SS']>ss_mu]
count=perm_kept['SS'].count()
pval=(count+1)/(1001)
pval

0.000999000999000999

In [17]:
perm_kept=perm_memPres[perm_memPres['OS']>ds_mu]
count=perm_kept['OS'].count()
pval=(count+1)/(1001)
pval

0.000999000999000999

In [5]:
ss_mu-ds_mu

0.15879464285714273

In [2]:
import pandas as pd
df = pd.DataFrame()
for i in range(9):
    tmp=pd.read_csv('/Users/aporter1350/Desktop/Porteretal_taskprediction/output/results/Ridge/ALL_Binary/fw/'+str(i)+'.csv', header= None).T
    tmp.rename(columns = {0:i},inplace=True)
    df=pd.concat([df, tmp], axis=1)
df.corr()

,0,1,2,3,4,5,6,7,8
0,1.000000,0.923832,0.969555,0.956661,0.956732,0.954334,0.918405,0.903154,0.957695
1,0.923832,1.000000,0.937011,0.926144,0.932250,0.922418,0.878331,0.853678,0.928220
2,0.969555,0.937011,1.000000,0.960973,0.965466,0.961844,0.935277,0.899900,0.960600
3,0.956661,0.926144,0.960973,1.000000,0.952774,0.954755,0.913052,0.899592,0.954179
4,0.956732,0.932250,0.965466,0.952774,1.000000,0.946766,0.917872,0.910823,0.959389
5,0.954334,0.922418,0.961844,0.954755,0.946766,1.000000,0.917023,0.897813,0.942658
6,0.918405,0.878331,0.935277,0.913052,0.917872,0.917023,1.000000,0.855924,0.914664
7,0.903154,0.853678,0.899900,0.899592,0.910823,0.897813,0.855924,1.000000,0.896715
8,0.957695,0.928220,0.960600,0.954179,0.959389,0.942658,0.914664,0.896715,1.000000


In [3]:
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10', 'groupwise']
df = pd.DataFrame()

for i in subList:
    tmp = pd.read_csv('/Users/aporter1350/Desktop/Porteretal_taskprediction/output/results/Ridge/ALL_Binary/fw/'+i+'.csv', header= None).T
    tmp.rename(columns = {0:i},inplace=True)
    df=pd.concat([df, tmp], axis=1)
    

    

In [31]:
mean(corrlist)

0.9287910556419903

In [33]:
stdev(corrlist)

0.02988163137019961

In [4]:
df.corr()
#color heatmap summary stat avg within vs between 

#feature weights across task within sub 

,MSC01,MSC02,MSC03,MSC04,MSC05,MSC06,MSC07,MSC10,groupwise
MSC01,1.000000,0.330177,0.354102,0.409090,0.320144,0.296402,0.412522,0.157283,0.297766
MSC02,0.330177,1.000000,0.242298,0.245388,0.193522,0.266015,0.236102,0.170836,0.149688
MSC03,0.354102,0.242298,1.000000,0.332120,0.220079,0.315464,0.302433,0.225541,0.148921
MSC04,0.409090,0.245388,0.332120,1.000000,0.286230,0.339159,0.169359,0.227877,0.265370
MSC05,0.320144,0.193522,0.220079,0.286230,1.000000,0.305180,0.258171,0.277559,0.291208
MSC06,0.296402,0.266015,0.315464,0.339159,0.305180,1.000000,0.287244,0.287279,0.272735
MSC07,0.412522,0.236102,0.302433,0.169359,0.258171,0.287244,1.000000,0.256880,0.110801
MSC10,0.157283,0.170836,0.225541,0.227877,0.277559,0.287279,0.256880,1.000000,0.259197
groupwise,0.297766,0.149688,0.148921,0.265370,0.291208,0.272735,0.110801,0.259197,1.000000


In [4]:
import pandas as pd
import os
import sys
thisDir = os.path.expanduser('~/Desktop/Porteretal_taskprediction/output/results/')
incremental = pd.read_csv(thisDir +'Ridge/single_task/incremental_groupavg.csv')
incremental['samples']= incremental['sample_per_sub']*7 * 2 # task and rest 7 subjects 


single_task=pd.read_csv(thisDir+'Ridge/single_task/acc.csv',usecols=['train_sub','task','same_sub','diff_sub'])
single_task=pd.melt(single_task,id_vars=['train_sub','task'],value_vars=['same_sub','diff_sub'],var_name='Analysis',value_name='acc')


In [23]:
CV=single_task[single_task['Analysis']=='same_sub'].groupby('task').mean()

In [27]:
comp = incremental.merge(CV, on = 'task')

win = comp[comp['acc_x']> comp['acc_y']]

In [29]:
win.groupby('task').mean()

,acc_x,sample_per_sub,samples,acc_y
task,,,,
glass,0.880263,6.00,84.0,0.878472
mem,0.992014,6.25,87.5,0.985938
motor,0.987171,8.00,112.0,0.981250


In [10]:
#############################################################################
####Groupwise and GW max compared to standard individualized classifiers#####
#############################################################################
#all packages and dataframes
import numpy as np
#import classification
import os
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import colors as mcolors
from matplotlib.lines import Line2D
import matplotlib.patches as patches
%matplotlib inline 
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
taskList=['mem','glass','semantic','motor']
modelList=['Log','Ridge','SVM']
netList=['unassign','default', 'visual','fp','dan','van','salience','co','sm','sm-lat', 'auditory','pmn','pon']
feature=[14808,10824,8736, 4620,5264,3151, 484, 4060, 2375,316, 564,45,21]
netRoi=dict([('unassign',14808),('default', 10824),('visual',8736),('fp', 4620),('dan',5264),('van',3151),('salience', 484),('co', 4060),('sm', 2375),('sm-lat', 316),('auditory', 564),('pmn',45),('pon',21)])
thisDir = os.path.expanduser('~/Desktop/Porteretal_taskprediction/output/results/')
homeDir = os.path.expanduser('~/Desktop/Porteretal_taskprediction/')
figsDir = os.path.expanduser('~/Desktop/Porteretal_taskprediction/output/figures/')
sys.path.append(homeDir+'code/pyScripts/Predictions')

MC_matched=pd.read_csv(thisDir+'Ridge/ALL_MC/acc_matched.csv')
MC_matched=pd.melt(MC_matched, id_vars=['train_sub'], value_vars = ['same_sub', 'diff_sub'], var_name = "Analysis", value_name = 'acc')
group_MC_matched=pd.read_csv(thisDir+'Ridge/ALL_MC/groupwise_matched_acc.csv')
gm_MC=group_MC_matched.mean()[0]
allTask_matched = pd.read_csv(thisDir+'Ridge/ALL_Binary/matched_acc.csv')
allTask_matched=pd.melt(allTask_matched,id_vars=['train_sub'],value_vars=['same_sub','diff_sub'],var_name='Analysis',value_name='acc')
allTask_group=pd.read_csv(thisDir+'Ridge/ALL_Binary/groupwise_acc.csv')
mL_group=allTask_group.mean()
single_task_matched=pd.read_csv(thisDir+'Ridge/single_task/acc_matched_groupavg.csv',usecols=['train_sub','task','same_sub','diff_sub'])
single_task_matched=pd.melt(single_task_matched,id_vars=['train_sub','task'],value_vars=['same_sub','diff_sub'],var_name='Analysis',value_name='acc')
single_task_group=pd.read_csv(thisDir+'Ridge/single_task/groupwise_acc.csv')
single_task_group['Analysis']='group'
comp_single_task=single_task_matched.groupby(['task','Analysis']).mean()
comp_single_task.reset_index(inplace=True)
compare_group=pd.concat([comp_single_task, single_task_group])

incremental = pd.read_csv(thisDir +'Ridge/single_task/incremental_groupavg.csv')
incremental['samples']= incremental['sample_per_sub']*7 * 2 # task and rest 7 subjects 

LOSO_group =pd.read_csv(thisDir+'Ridge/single_task/groupwise_LOSO_acc.csv')
LOSO_group['Analysis'] = 'groupwiseMax'
compare_group_all = pd.concat([compare_group, LOSO_group])

gw_max_binary = pd.read_csv(thisDir +'Ridge/ALL_Binary/groupwiseMax_acc.csv')
gwMax = gw_max_binary.mean()[0]

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [30]:
SS_bin=allTask_matched[allTask_matched['Analysis']=='same_sub']

#SS_bin.mean()[0]-allTask_group.mean()[0] #groupwise 1 session per person
SS_bin.mean()[0]-gw_max_binary.mean()[0] #groupwise max


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  after removing the cwd from sys.path.


0.021401964662447148

In [28]:
SS_task=single_task_matched[single_task_matched['Analysis']=='same_sub']
SS_new=SS_task.groupby('task').mean()
LOSO_group

,task,acc,Analysis
0,glass,0.879934,groupwiseMax
1,semantic,0.912500,groupwiseMax
2,motor,0.987171,groupwiseMax
3,mem,1.000000,groupwiseMax


In [22]:
SS_new.reset_index(inplace=True)
final = SS_new.merge(single_task_group, on = ['task'])

In [26]:
final['diff']=final['acc_x'] - final['acc_y']

In [27]:
final

,task,acc_x,acc_y,Analysis,diff
0,glass,0.859375,0.562500,group,0.296875
1,mem,0.984375,0.890625,group,0.093750
2,motor,0.984375,0.828125,group,0.156250
3,semantic,0.921875,0.671875,group,0.250000


In [29]:

final_gwMax = SS_new.merge(LOSO_group, on = ['task'])
final_gwMax['diff']=final_gwMax['acc_x'] - final_gwMax['acc_y']
final_gwMax

,task,acc_x,acc_y,Analysis,diff
0,glass,0.859375,0.879934,groupwiseMax,-0.020559
1,mem,0.984375,1.000000,groupwiseMax,-0.015625
2,motor,0.984375,0.987171,groupwiseMax,-0.002796
3,semantic,0.921875,0.912500,groupwiseMax,0.009375


In [48]:
#Inetworks permutation test 

ALL_Bin=pd.read_csv(outDir+'ALL_Binary/iNets_acc.csv', usecols=[1,2])
ALL_MC=pd.read_csv(outDir+'ALL_MC/iNets_acc_matched.csv', usecols=[1,2])
mixed=pd.read_csv(outDir+'single_task/mixed_iNets_acc_matched.csv',usecols=[1,2])
slow=pd.read_csv(outDir+'single_task/slowreveal_iNets_acc_matched.csv',usecols=[1,2])

In [49]:
Diff = ALL_Bin[ALL_Bin['Analysis']=='SS'].acc.values-ALL_Bin[ALL_Bin['Analysis']=='OS'].acc.values
ALL_Diff=mean(Diff)

distribution=[]
for i in range(1000):
    null=shuffle(ALL_Bin)
    distribution.append(null)
null=pd.DataFrame(distribution,columns=['distribution'])

keep=null[null['distribution']>ALL_Diff]
count=keep['distribution'].count()
p=(count+1)/1001
p

0.000999000999000999

In [50]:
ALL_Diff

0.35113039113030203

In [51]:
Diff = ALL_MC[ALL_MC['Analysis']=='SS'].acc.values-ALL_MC[ALL_MC['Analysis']=='OS'].acc.values
ALL_Diff=mean(Diff)

distribution=[]
for i in range(1000):
    null=shuffle(ALL_MC)
    distribution.append(null)
null=pd.DataFrame(distribution,columns=['distribution'])

keep=null[null['distribution']>ALL_Diff]
count=keep['distribution'].count()
p=(count+1)/1001
p

0.000999000999000999

In [52]:
ALL_Diff

0.30143474261610287

In [53]:
Diff = mixed[mixed['Analysis']=='SS'].acc.values-mixed[mixed['Analysis']=='OS'].acc.values
ALL_Diff=mean(Diff)

distribution=[]
for i in range(1000):
    null=shuffle(mixed)
    distribution.append(null)
null=pd.DataFrame(distribution,columns=['distribution'])

keep=null[null['distribution']>ALL_Diff]
count=keep['distribution'].count()
p=(count+1)/1001
p

0.000999000999000999

In [54]:
ALL_Diff

0.2889874840041764

In [55]:
Diff = slow[slow['Analysis']=='SS'].acc.values-slow[slow['Analysis']=='OS'].acc.values
ALL_Diff=mean(Diff)

distribution=[]
for i in range(1000):
    null=shuffle(slow)
    distribution.append(null)
null=pd.DataFrame(distribution,columns=['distribution'])

keep=null[null['distribution']>ALL_Diff]
count=keep['distribution'].count()
p=(count+1)/1001
p

0.000999000999000999

In [58]:
null

,distribution
0,-0.048653
1,-0.015282
2,0.032135
3,0.062054
4,0.024911
...,...
995,-0.054087
996,0.004287
997,0.018833
998,0.043791
